In [ ]:
# default_exp signIn

# SignIn

> Log into account

In [1]:
#hide
import src.setEnv 

In [2]:
#export 
from src.userTable import UserTable
from awsSchema.apigateway import Response, Event
from copy import deepcopy
from src.saltHashPassword import hash_password, check_password
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from beartype import beartype

## Sign In Input Class

In [37]:
#export 
@dataclass_json
@dataclass
class SignInInput:
    username: str
    password: str
    
    def countUsername(self):
        count = UserTable.username_index.count(self.username)
        if count:
            return True
        else:
            return False
    
    def checkPassword(self):
        for item in UserTable.username_index.query(self.username):
            return check_password(item.passwordHash, self.password)

## Main Lambda Handler

In [41]:
#export
class H:
    class ParseInputError(Exception): pass
    class CountUsernameError(Exception): pass
    class InvalidUsernameError(Exception): pass
    class PasswordCheckError(Exception): pass
    @classmethod
    @beartype
    def parseInput(cls, event: dict)->SignInInput:
        try:
            user = Event.parseDataClass(SignInInput, deepcopy(event))
            return user
        except Exception as e:
            raise cls.ParseInputError(e)
    
    @classmethod
    @beartype
    def checkLoginDetails(cls, user:SignInInput):
        try:
            count = user.countUsername
        except Exception as e:
            raise cls.CountUsernameError(e)
        if count:
            try:
                passwordChecked = user.checkPassword()
            except Exception as e:
                raise cls.PasswordCheckError(e)
            if passwordChecked:
                return True
            else:
                raise cls.InvalidUsernameError

In [42]:
#export
def signIn(event, *args):
    try:
        user = H.parseInput(event)
        H.checkLoginDetails(user)
        return Response.returnSuccess()
    except H.ParseInputError as e:
        return Response.returnError(f'failed to parse input {e}')
    except H.CountUsernameError as e:
        return Response.returnError(f'failed to count number of user with specified username {e}')
    except H.InvalidUsernameError as e:
        return Response.returnError(f'incorrect username or password {e}')
    except H.PasswordCheckError as e:
        return Response.returnError(f'failed to check username and password {e}')
    except Exception as e:
        return Response.returnError(f' unknown error {e}')

In [43]:
input_ = {'username': 'Delgg4', 'password': '12765'}
event = Event.getInput(body = input_)
signIn(event)

{'body': '{}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}